# Trabalho prático 2
### Luis Gustavo Lorgus Decker
### Luiz Antonio Falaguasta Barbosa

Importando as bibliotecas utilizadas

In [1]:
import numpy as np #Tratamento numérico
import matplotlib.pyplot as plt #Plot de gráficos
from sklearn import linear_model #Regressão linear
from sklearn import decomposition #Decomposição PCA
from sklearn import neural_network
import PIL.Image as Image#Abrir imagens
import sklearn.metrics#Metricas de avaliação
from math import floor

Função auxiliar para truncamento

In [2]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

Carregamos dados de treinamento
          
    Carregamos no vetor pixelsExpanded os tres canais de cada imagem.
    Cada elemento de 'pixelsExpanded' corresponde a uma imagem, e é uma matriz de 32x32 pixeis(tamanho da imagem)

In [3]:
#str(100).zfill(5)
#Ler arquivo de labels e parear com images.
labels = np.loadtxt(fname='cifar-10/train/labels',unpack=True)
#Ler as imagens
#As amostras estão nas linhas, as features estão nas colunas
datasetSize = 50000
nImages = 20000#Numero de imagens a serem carregadas(ATENÇÃO, MODIFICAR SEPARAÇÃO DE DADOS DE VALIDAÇÃO)
pixelsExpanded = np.empty((nImages,3073),int)#Cria a matriz de dados, com nImages linhas e 3073 colunas (uma para o label e 3072 para os pixeis rgb)
for img in range(0,nImages):#Para cada imagem 
    pixelsExpanded[img][0] = labels[img] #Atribui o label a primeira coluna
    imageIndex = str(img).zfill(5)
    filename = './cifar-10/train/' + imageIndex + '.png'#Cria o filename
    image = Image.open(fp=filename)#Carrega a imagem
    #print(image.size,image.getbands())
    pixels = image.getdata()#Acessa os pixeis
    #print(len(pixels))
    col = 1;#Indice atual do primeiro dado
    #Para cada imagem, ler os canais RGB e anexar a matrix de dados
    for pixel in range(0,len(pixels)):
        
        r,g,b = pixels[pixel][0:3] #Separa os canais
        #Adiciona o pixel
        pixelsExpanded[img][col] =r 
        pixelsExpanded[img][col+1] = g 
        pixelsExpanded[img][col+2] = b
        #print("Tratado imagem {} , pixel {} , ultimo indice {}".format(img,pixel,col+2))
        col = col+3 #Avança 3 posições
    #print("Processed {}% of the dataset".format(img/datasetSize))
print("Lidos os pixeis de {} imagens".format(nImages))

    

Lidos os pixeis de 20000 imagens


Separamos **80%** das imagens como conjunto de treino, e **20%** como conjunto de validação.

In [4]:
trainingSetSize = floor(nImages*0.8) #Tamanho desejado do conjunto de treinamento
trainingSet = pixelsExpanded[0:trainingSetSize][:] #Separa os trainingSetSize primeiros dados como treinamento
validationSetSize = nImages - trainingSetSize #Tamanho do conjunto de Validação
validationSet = pixelsExpanded[trainingSetSize:][:] #Separa os validationSetSize dados após o ultimo usado em training set como validação.
del pixelsExpanded


Aplicar o **PCA** nos dados

    A função PCA abaixo realiza uma redução linear de dimensionalidade utilizando uma decomposição de valores singulares (SVD). Assim, esta redução projeta os dados em um espaço de menor dimensionalidade.
    
    O SVD é realizado em cima da matriz de covariância dos dados, e retorna três matrizes:
    
    -A matriz U, que contém os autovetores dos quais escolhemos k que descrevem o hiperplano em Rk sob qual os dados serão projetados. Este k é escolhido de tal maneira que 'preservedVariance' seja a percentagem da variância dos dados preservada.
    
    -A matriz S, que contém os autovalores relacionados aos autovetores da matriz U. Estes autovetores representam a variância preservada em cada autovetor associado. Ou seja, escolhemos k autovetores de tal maneira que a soma de suas variancias sobre a soma da variancia total seja igual a 'preservedVariance'
    
    -A matriz V, que não utilizamos
    
    Utilizamos como método de calculo do SVD 'full', que realiza a decomposição completa. 
   

In [5]:
preservedVariance = 0.95
pca = decomposition.PCA(n_components=preservedVariance,svd_solver='full')
trainedPca = pca.fit(trainingSet[:,1:3072])
#ATENCAO
#Temos que guardar o pca treinado para executar a mesma
#redução de dimensionalidade no conjunto de validaçao/teste

Reduzimos a dimensionalidade do conjunto de treino

In [6]:
trainingSetReduced = trainedPca.transform(trainingSet[:,1:3072])
#print(trainingSetReduced.shape)
nFeatures = trainingSetReduced.shape[1]
print(nFeatures)


213


Reduzimos também a dimensionalidade do conjunto de validação

In [16]:
validationSetReduced = trainedPca.transform(validationSet[:,1:3072])

Para analisar o resultado e a eficiência do PCA, rodamos o algorítmo mantendo 99% da variância, 95% e 90%. 

**Para 99%**, reduzimos para 656 dimensões (2416 dimensões a menos).

**Para 95%**, reduzimos para 216 dimensões (2856 dimensões a menos).

**Para 90%**, reduzimos 99 para  dimensões (2973 dimensões a menos).

Escolhemos então seguir preservando 95% da variância, com uma redução de ~92% das dimensões.

Separamos também os dados das images de suas respectivas labels

In [17]:
trainlLabels = trainingSet[:,0]#Separa os labels do conjunto de treino
trainData = trainingSetReduced#Separa os dados do conjunto de treino
del trainingSetReduced
validationLabels = validationSet[:,0]#Separa os labels do conjunto de validação
validationData = validationSetReduced#Separa os dados do conjunto de validação




NameError: name 'trainingSetReduced' is not defined

### Inicializamos a regressão linear.

    Para classificação multiclasses, o algorítmo utiliza a metodologia "um vs resto" (parametro multi_class).

    O parâmetro C é o inverso da força de regularização. Valores menores implicam uma regularização mais forte. 

    Esta função também adiciona uma constante (bias) a função de decisão, permitindo um deslocamento na interceptação do eixo Y.

    Como método de regularização, utiliza L2-normalization. Neste método as penalizações aplicadas durante a atualização dos coeficientes é 
$$\begin{align}
\lambda * \sum_{i=1}^{k} ( \theta_i^2 )
\end{align}$$

    ou seja, a soma dos quadrados dos pesos. 
    
    
    Como função de minimização de custos, o classificador utiliza uma técnica de 'Coordinate descent', uma variante do gradiente descendente.
    
    max_iter especifica o número máximo de iterações que serão feitas visando minimizar a função de custo.


In [9]:
iteracoes = 100
forca_regularizacao = 1
logreg=linear_model.LogisticRegression(C=forca_regularizacao,multi_class='ovr',max_iter=iteracoes)

Treinamos o classificador com os dados de treinamento

In [10]:
logreg = logreg.fit(trainData,trainlLabels.astype('int'))

Aplicamos a regressão nos dados de validação

In [11]:
classified = logreg.predict(validationSetReduced)

Calculamos a percentagem de acertos

In [12]:
hits = truncate((sklearn.metrics.accuracy_score(validationLabels,classified)*100),2)
print("Acertou {}% dos exemplos".format(hits))

Acertou 39.20% dos exemplos


#### Agora, treinamos uma regressão logística multinomial(softmax), utilizando *cross-entropy*.

    Como função de minimização, utilizamos o Stochastic Average Gradient. 

In [13]:
logreg=linear_model.LogisticRegression(C=forca_regularizacao,multi_class='multinomial',max_iter=iteracoes,solver='sag')

Treinamos com os dados de treinamento

In [14]:
logreg = logreg.fit(trainData,trainlLabels.astype('int'))

Aplicamos nos dados de validação

In [15]:
classified = logreg.predict(validationSetReduced)

Calculamos a percentagem de acertos

In [16]:
hits = truncate((sklearn.metrics.accuracy_score(validationLabels,classified)*100),2)
print("Acertou {}% dos exemplos".format(hits))

Acertou 39.17% dos exemplos


### Iniciamos Redes Neurais

In [17]:
Solver = 'sgd' #Funcao de minimização. 
HiddenLayers = (trainData.shape[1],) #Numero de neurônios na camada escondida
ActivationFunction = ['logistic','tanh','relu']#Funcoes de ativação
BatchSize = 200 #Numero de amostras em cada batch
LearningRate = 'adaptive' #Metodo de learning rate. Adaptive divide por 5 assim que duas epocas consecutivas 
#falharem em reduzir o custo.
Iterations = 100 #Numero de iteracões maxima



Treinamos três redes neurais, cada uma com uma função de ativação

In [ ]:
networks= []
for func in ActivationFunction:
    NeuralNetwork = neural_network.MLPClassifier(solver=Solver, activation=func, hidden_layer_sizes=HiddenLayers, batch_size= BatchSize, learning_rate = LearningRate, max_iter= Iterations)
    NeuralNetwork = NeuralNetwork.fit(trainData,trainlLabels.astype('int'))
    networks.append(NeuralNetwork)
    

Realizamos a classificação

In [19]:
networksResults = []
for net in networks:
    result = net.predict(validationSetReduced)
    result = truncate((sklearn.metrics.accuracy_score(validationLabels,result)*100),2)
    networksResults.append(result)


Resultado das redes, com cada tipo de função de ativação

In [20]:
index = 0
for i in ActivationFunction:
    print ("Com a função de ativação {}, acertou {}% dos exemplos".format(i,networksResults[index]))
    index = index + 1

Com a função de ativação logistic, acertou 39.55% dos exemplos
Com a função de ativação tanh, acertou 37.37% dos exemplos
Com a função de ativação relu, acertou 41.47% dos exemplos


In [21]:
Iterations = 200 #Numero de iteracões maxima

In [23]:
networks= []
for func in ActivationFunction:
    NeuralNetwork = neural_network.MLPClassifier(solver=Solver, activation=func, hidden_layer_sizes=HiddenLayers, batch_size= BatchSize, learning_rate = LearningRate, max_iter= Iterations)
    NeuralNetwork = NeuralNetwork.fit(trainData,trainlLabels.astype('int'))
    networks.append(NeuralNetwork)
    
networksResults = []
for net in networks:
    result = net.predict(validationSetReduced)
    result = truncate((sklearn.metrics.accuracy_score(validationLabels,result)*100),2)
    networksResults.append(result)
index = 0
winner = 0
winnerScore = 0
for i in ActivationFunction:
    print ("Com a função de ativação {}, acertou {}% dos exemplos".format(i,networksResults[index]))

    index = index + 1

/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Com a função de ativação logistic, acertou 39.32% dos exemplos
Com a função de ativação tanh, acertou 36.77% dos exemplos
Com a função de ativação relu, acertou 39.72% dos exemplos


Agora, rodamos novamente com duas camadas ocultas

In [24]:
networks= []
HiddenLayers = (trainData.shape[1],trainData.shape[1])
for func in ActivationFunction:
    NeuralNetwork = neural_network.MLPClassifier(solver=Solver, activation=func, hidden_layer_sizes=HiddenLayers, batch_size= BatchSize, learning_rate = LearningRate, max_iter= Iterations)
    NeuralNetwork = NeuralNetwork.fit(trainData,trainlLabels.astype('int'))
    networks.append(NeuralNetwork)
    
networksResults = []
for net in networks:
    result = net.predict(validationSetReduced)
    result = truncate((sklearn.metrics.accuracy_score(validationLabels,result)*100),2)
    networksResults.append(result)
index = 0
winner = 0
winnerScore = 0
for i in ActivationFunction:
    print ("Com a função de ativação {}, acertou {}% dos exemplos".format(i,networksResults[index]))

    index = index + 1

/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Com a função de ativação logistic, acertou 38.75% dos exemplos
Com a função de ativação tanh, acertou 37.50% dos exemplos
Com a função de ativação relu, acertou 9.82% dos exemplos


#### Teste final
Carregamos os arquivos de teste

In [7]:
#str(100).zfill(5)
#Ler arquivo de labels e parear com images.
labels = np.loadtxt(fname='cifar-10/train/labels',unpack=True)
#Ler as imagens
#As amostras estão nas linhas, as features estão nas colunas
datasetSize = 10000
nImages = datasetSize#Numero de imagens a serem carregadas(ATENÇÃO, MODIFICAR SEPARAÇÃO DE DADOS DE VALIDAÇÃO)
pixelsExpanded = np.empty((nImages,3073),int)#Cria a matriz de dados, com nImages linhas e 3073 colunas (uma para o label e 3072 para os pixeis rgb)
for img in range(0,nImages):#Para cada imagem 
    pixelsExpanded[img][0] = labels[img] #Atribui o label a primeira coluna
    imageIndex = str(img).zfill(5)
    filename = './cifar-10/test/' + imageIndex + '.png'#Cria o filename
    image = Image.open(fp=filename)#Carrega a imagem
    #print(image.size,image.getbands())
    pixels = image.getdata()#Acessa os pixeis
    #print(len(pixels))
    col = 1;#Indice atual do primeiro dado
    #Para cada imagem, ler os canais RGB e anexar a matrix de dados
    for pixel in range(0,len(pixels)):
        
        r,g,b = pixels[pixel][0:3] #Separa os canais
        #Adiciona o pixel
        pixelsExpanded[img][col] =r 
        pixelsExpanded[img][col+1] = g 
        pixelsExpanded[img][col+2] = b
        #print("Tratado imagem {} , pixel {} , ultimo indice {}".format(img,pixel,col+2))
        col = col+3 #Avança 3 posições
    #print("Processed {}% of the dataset".format(img/datasetSize))
print("Lidos os pixeis de {} imagens".format(nImages))

Lidos os pixeis de 10000 imagens


Reduz a dimensionalidade dos dados usando o mesmo PCA treinado para o conjunto de treino

In [8]:
testSetReduced = trainedPca.transform(pixelsExpanded[:,1:3072])
testLabels = pixelsExpanded[:,0]

Treinamos com o conjunto de treino inteiro

In [14]:
fullTrainSet = trainingSetReduced
fullTrainLabel = trainData

NameError: name 'trainData' is not defined

In [13]:
Solver = 'sgd' #Funcao de minimização. 
HiddenLayers = (fullTrainLabel.shape[1],) #Numero de neurônios na camada escondida
ActivationFunction = 'relu' #Funcoes de ativação
BatchSize = 200 #Numero de amostras em cada batch
LearningRate = 'adaptive' #Metodo de learning rate. Adaptive divide por 5 assim que duas epocas consecutivas 
#falharem em reduzir o custo.
Iterations = 100 #Numero de iteracões maxima
network = neural_network.MLPClassifier(solver=Solver, activation=func, hidden_layer_sizes=HiddenLayers, batch_size= BatchSize, learning_rate = LearningRate, max_iter= Iterations)
networks[0].fit(fullTrainData,fullTrainSet)
netResult = networks[winner].predict(testSetReduced)
result = truncate((sklearn.metrics.accuracy_score(testLabels,netResult)*100),2)
print ("Resultado final: {}%".format(result))

NameError: name 'fullTrainLabel' is not defined

## Printamos a matriz de confusão

In [ ]:
labels = ['0 airplane', '1 automobile', '2 bird', '3 cat', '4 deer', '5 dog', '6 frog', '7 horse', '8 ship', '9 truck']
cm = confusion_matrix(testLabels, netResult, labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
pl.title('Matriz de confusão do classificador')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
pl.xlabel('Predito')
pl.ylabel('Real')
pl.show()